In [ ]:
import tenseal as ts
import numpy as np
import math

import matplotlib.pyplot as plt

Error: Session cannot generate requests

In [43]:
# SEAL Context
bits_scale = 26
    
# Create TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[31, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, 31]
)

context.global_scale = pow(2, bits_scale)
context.generate_galois_keys()

In [54]:
# minimal context
bits_scale = 30
mini_context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[40, bits_scale, 40]
)
mini_context.global_scale = pow(2,bits_scale)
mini_context.generate_galois_keys()

In [55]:
N = 100

x_test = np.linspace(0.1,0.9,N) + np.random.uniform(-0.1, 0.1, N)

x_enc = ts.ckks_vector(context, x_test)

result = x_enc.decrypt()

In [56]:
# error depending on vector size
# result: independent of vector size

rounds = 100

for i in range(int(math.log2(4096))):
    N = pow(2,i)
    ones = np.ones(N)

    cum_mean = 0
    for _ in range(rounds):
        ones_enc = ts.ckks_vector(mini_context, ones)
        ones_dec = ones_enc.decrypt()
        cum_mean += np.abs(ones_dec - ones).mean()

    print(f"N = {N}, mean err: {cum_mean/rounds}")

N = 1, mean err: 7.526121966361376e-07
N = 2, mean err: 8.852880563448018e-07
N = 4, mean err: 7.755679233523161e-07
N = 8, mean err: 8.478355722757458e-07
N = 16, mean err: 9.400409964475942e-07
N = 32, mean err: 8.361771451375482e-07
N = 64, mean err: 8.751137455512842e-07
N = 128, mean err: 8.91431907156827e-07
N = 256, mean err: 9.064474561546821e-07
N = 512, mean err: 9.128357340432483e-07
N = 1024, mean err: 9.003657933677087e-07
N = 2048, mean err: 8.982327885791072e-07


In [81]:
# error depending on Q
# result: independent

bits_scale = 26
ones = np.ones(1000)
zeros = np.zeros(1000)

for factors in range(7):
    context = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=8192,
        coeff_mod_bit_sizes=[31] + [bits_scale]*factors + [31]
    )
    context.global_scale = 2**bits_scale
    context.generate_galois_keys()

    cum_mean = 0
    cum_err = 0
    for _ in range(rounds):
        ones_enc = ts.ckks_vector(context, ones)
        ones_dec = ones_enc.decrypt()
        cum_mean += np.abs(ones_dec - ones).mean()

        zeros_enc = ts.ckks_vector(context, zeros)
        cum_err += np.abs(zeros_enc.decrypt()).mean()

    print(f"factors = {factors} ({2*31+factors*bits_scale}), mean err: {cum_mean/rounds}, {cum_err/rounds}")

factors = 0 (62), mean err: 1.4490454069258007e-05, 1.456294624597553e-05
factors = 1 (88), mean err: 1.4307343531411287e-05, 1.431391443885612e-05
factors = 2 (114), mean err: 1.4599856567936198e-05, 1.457127863534937e-05
factors = 3 (140), mean err: 1.4371146184657268e-05, 1.4356613168565623e-05
factors = 4 (166), mean err: 1.4448980582386583e-05, 1.4371659443952832e-05
factors = 5 (192), mean err: 1.4846788531525106e-05, 1.4862932361023368e-05
factors = 6 (218), mean err: 1.4400299142292786e-05, 1.4464743642083273e-05


In [83]:
# error depending on N
# result: 
rounds = 100
ones = np.ones(1000)
zeros = np.zeros(1000)

for power in range(12,16):
    context = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=2**power,
        coeff_mod_bit_sizes=[40,28,40]
    )
    context.global_scale = pow(2,35)
    context.generate_galois_keys()

    ones = np.ones(2**(power-1))

    cum_mean = [0,0]
    for _ in range(rounds):
        ones_enc = ts.ckks_vector(context, ones)
        ones_dec = ones_enc.decrypt()
        cum_mean[0] += np.abs(ones_dec - ones).mean()

        zeros_enc = ts.ckks_vector(context, zeros)
        cum_mean[1] += np.abs(zeros_enc.decrypt()).mean()

    print(f"N = {2**power}, mean err: {cum_mean[0]/rounds}, {cum_mean[1]/rounds}")

N = 4096, mean err: 1.3976430293565281e-08, 1.4011314666841292e-08
N = 8192, mean err: 2.8284831483558712e-08, 2.808657132738133e-08
N = 16384, mean err: 5.591070808325018e-08, 5.755099980945862e-08
N = 32768, mean err: 1.1227329613539743e-07, 1.1224921566902248e-07


In [85]:
# error depending on scale
# result: 
rounds = 100
ones = np.ones(1000)

for bits_scale in range(20,41):
    context = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=4096,
        coeff_mod_bit_sizes=[bits_scale+2,bits_scale+2]
    )
    context.global_scale = pow(2,bits_scale)
    context.generate_galois_keys()

    cum_mean = [0,0]
    for _ in range(rounds):
        ones_enc = ts.ckks_vector(context, ones)
        ones_dec = ones_enc.decrypt()
        cum_mean[0] += np.abs(ones_dec - ones).mean()

        zeros_enc = ts.ckks_vector(context, zeros)
        cum_mean[1] += np.abs(zeros_enc.decrypt()).mean()

    print(f"scale = {bits_scale}, mean err: {cum_mean[0]/rounds}, {cum_mean[1]/rounds}")

scale = 20, mean err: 0.0004600849456141923, 0.000459079393921872
scale = 21, mean err: 0.00022897189663774497, 0.00022842234780902872
scale = 22, mean err: 0.00011574096328215578, 0.00011547308648206555
scale = 23, mean err: 5.779782011234685e-05, 5.764629531064816e-05
scale = 24, mean err: 2.8757916740253435e-05, 2.8834825276405593e-05
scale = 25, mean err: 1.4677669651701036e-05, 1.4597299592229113e-05
scale = 26, mean err: 7.3105883794934876e-06, 7.278629977414044e-06
scale = 27, mean err: 3.646272661822824e-06, 3.661231885859972e-06
scale = 28, mean err: 1.815913714239526e-06, 1.8129870872689068e-06
scale = 29, mean err: 9.244405772471532e-07, 9.23740753678079e-07
scale = 30, mean err: 4.500813111556456e-07, 4.520433771529676e-07
scale = 31, mean err: 2.2669142147112242e-07, 2.2479866642825063e-07
scale = 32, mean err: 1.1181236042956403e-07, 1.1129498520205299e-07
scale = 33, mean err: 5.5812952763503186e-08, 5.588936806591887e-08
scale = 34, mean err: 2.82593436164269e-08, 2.827

In [144]:
# plot errors vs. N and scale
rounds = 100

n = list(range(13,15))
bit_scales = list(range(20,51))
e = []

for n_i in n:
    N = 2**n_i
    zeros = np.zeros(int(N/2))
    print(f"N = {N}")

    errors = []

    for bits_scale in bit_scales:
        context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=N,
            coeff_mod_bit_sizes=[54,54]
        )
        context.global_scale = pow(2,bits_scale)
        context.generate_galois_keys()

        cum_mean = 0
        for _ in range(rounds):
            zeros_enc = ts.ckks_vector(context, zeros)
            cum_mean += np.abs(zeros_enc.decrypt()).mean()

        # print(f"scale = {bits_scale}, mean err: {cum_mean/rounds}")
        errors.append(cum_mean/rounds)

    e.append(errors)



N = 4096
N = 8192
N = 16384
N = 32768


In [ ]:
from matplotlib import ticker, cm

fig, ax = plt.subplots(figsize=(10,6))

cf = ax.contourf(bit_scales,n,e,alpha=0.75,locator=ticker.LogLocator())
fig.colorbar(cf, ax=ax, format=ticker.LogFormatter(10))

# plt.contourf(bit_scales, n, e)
plt.yticks(n)
plt.ylabel(f"log $N$")
plt.xlabel(f"bits_scale")
plt.title("Error after en- & decrypt")
plt.grid()
plt.show()

Error: Session cannot generate requests

In [ ]:
bit_scales = list(range(20,60))
# zeros = np.zeros(8192//2)

for bits_scale in bit_scales:
        context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=8192,
            coeff_mod_bit_sizes=[60,60]
        )
        context.global_scale = pow(2,bits_scale)
        context.generate_galois_keys()

        cum_mean = 0
        for _ in range(rounds):
            zeros_enc = ts.ckks_vector(context, zeros)
            cum_mean += np.abs(zeros_enc.decrypt()).mean()

        print(f"scale = {bits_scale}, mean err: {cum_mean/rounds}")

Error: Session cannot generate requests